# Importando e tratando a base de dados

## Importação dos dados

In [ ]:
import json
import requests
from io import StringIO
import pandas as pd
import plotly.express as px

In [ ]:
with open("/home/daniel/geohmount/code/credentials.json", "r") as credentials_file:
    credentials = json.load(credentials_file)

owner = credentials["github"]["username"]
token = credentials["github"]["token"]
repo = credentials["github"]["repo"]
path = 'trajs/sb/resultado.csv'

r = requests.get(f'https://api.github.com/repos/{owner}/{repo}/contents/{path}',
    headers={
        'accept': 'application/vnd.github.v3.raw',
        'authorization': f'token {token}'
            }
                )

string_io_obj = StringIO(r.text)

In [ ]:
trajs = pd.read_csv(string_io_obj, sep=";")

In [ ]:
trajs.head()

## Tratamento dos dados

### Renomeando colunas

In [ ]:
trajs = trajs.rename(columns={'velocidade_km/h': 'velocidade', 'rain': 'chuva_gdas', 'evt_sum_rain': 'evt_soma_chuva', 
                              'evt_avg_rain': 'evt_media_chuva', 'evt_max_rain': 'evt_max_chuva', 'direcao_graus': 'direcao'})

In [ ]:
trajs = trajs.dropna()
trajs.head()

### Tipando a coluna de data

In [ ]:
trajs['data'] = pd.to_datetime(trajs['data'])

### Novas colunas

#### Sentidos das trajetórias

In [ ]:
direcoes = ("N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW")

In [ ]:
def direcao_categorica(direcao_em_graus):
    indice = int(direcao_em_graus // 22.5)
    resto = direcao_em_graus % 22.5
    if resto > 11.25:
        indice += 1
    if indice == 16:
        indice = 0
    return direcoes[indice]

In [ ]:
trajs['direcao_cat'] = trajs['direcao'].apply(direcao_categorica)
trajs.head(10)

#### Categorizando as velocidades

In [ ]:
velocidades = ('0-5 km/h', '5-10 km/h', '10-15 km/h', '15-20 km/h', '20-25 km/h', '25-30 km/h', '30-35 km/h', '35-40 km/h','40-45 km/h')

In [ ]:
def velocidade_categorica(velocidade_kmh):
    indice = int(velocidade_kmh // 5)
    return velocidades[indice]

In [ ]:
trajs['velocidade_cat'] = trajs['velocidade'].apply(velocidade_categorica)
trajs.head(10)

#### Categorizando a soma de chuva por evento

In [ ]:
chuvas = ('5-20 mm', '20-35 mm', '35-50 mm', '50-65 mm', '65-80 mm', '80-95 mm', '95-110 mm', '110-125 mm', '125-130 mm')

In [ ]:
def chuva_categorica(chuva_mm):
    chuva_mm -= 5
    indice = int(chuva_mm // 15)
    return chuvas[indice]

In [ ]:
trajs['evt_soma_chuva_cat'] = trajs['evt_soma_chuva'].apply(chuva_categorica)
trajs.head(10)

### Subsets por altitude

In [ ]:
trajs_509 = trajs.query("altitude == 509")
trajs_659 = trajs.query("altitude == 659")
trajs_909 = trajs.query("altitude == 909")

# Criando os gráficos

## Definindo as funções

In [ ]:
def vel_plot(dataframe, title='<b>Distribuição das massas de ar resultantes em chuva no Soberbo</b>', template='plotly'):
    
    color_map = dict(zip(velocidades, px.colors.sequential.Plasma_r))
    
    fig = px.bar_polar(dataframe, r='frequencia', theta='direcao_cat', color='velocidade_cat', color_discrete_map=color_map,
                       category_orders={'velocidade_cat': velocidades, 'direcao_cat': direcoes}, 
                       color_discrete_sequence = px.colors.sequential.Plasma_r)

    fig.update_layout(legend=dict(traceorder='reversed', 
                                  yanchor='top', 
                                  y=0.99, 
                                  xanchor='right', 
                                  x=1.1, 
                                  title_font_size=13), 
                      legend_title_text='<b>Velocidade</b>', 
                      title={'text': title, 'y': 0.97, 'x': 0.5, 'xref': 'paper'}, 
                      font_family='Open Sans', 
                      annotations=[dict(
                          showarrow=False, 
                          xref='paper', 
                          yref='paper', 
                          x=0.01, y=0.95, 
                          text='<b>Altitude: 659m',
                          font=dict(size=14, color='black'))], 
                      polar_angularaxis_tickfont_size = 15,
                      template=template
                     )

    fig.update_traces(hovertemplate='<b>%{theta}</b><br>' + 'frequência: %{r:.1f}')
    
    return fig

In [ ]:
def chuva_plot(dataframe, title='<b>Distribuição das massas de ar resultantes em chuva no Soberbo</b>', 
               template='plotly', annotation_color='black'):
    
    color_map = dict(zip(chuvas, px.colors.sequential.Blues))
    
    fig = px.bar_polar(dataframe, r='frequencia', theta='direcao_cat', color='evt_soma_chuva_cat', color_discrete_map=color_map,
                       category_orders={'evt_soma_chuva_cat': chuvas, 'direcao_cat': direcoes})

    fig.update_layout(legend=dict(traceorder='reversed', 
                                  yanchor='top', 
                                  y=0.99, 
                                  xanchor='right', 
                                  x=1.1, 
                                  title_font_size=14), 
                      legend_title_text='<b>Precipitação</b>', 
                      title={'text': title, 'y': 0.98, 'x': 0.5, 'xref': 'paper'}, 
                      font_family='Open Sans', 
                      polar_radialaxis_showticklabels=False, 
                      annotations=[dict(
                          showarrow=False, 
                          xref='paper', 
                          yref='paper', 
                          x=0.01, y=0.95, 
                          text='<b>Altitude: 659m', 
                          font=dict(size=14, color=annotation_color))], 
                      polar_angularaxis_tickfont_size = 15, 
                      template=template
                     )

    fig.update_traces(hovertemplate='<b>%{theta}</b><br>' + 'frequencia: %{r:.1f}')

    return fig

## Geral

### Velocidade

In [ ]:
gp_vel = trajs_659.groupby(['direcao_cat', 'velocidade_cat']).size().reset_index(name='frequencia')
gp_vel.head(5)

In [ ]:
fig = vel_plot(gp_vel)
fig.show()

### Chuva

In [ ]:
gp_chuva = trajs_659.groupby(['direcao_cat', 'evt_soma_chuva_cat']).size().reset_index(name='frequencia')
gp_chuva.head(5)

In [ ]:
fig = chuva_plot(gp_chuva)
fig.show()

## Estação úmida/seca

### Dataframes

In [ ]:
meses_chuvosos = ('Jan', 'Feb', 'Mar', 'Apr', 'Oct', 'Nov', 'Dec')
meses_secos = ('May', 'Jun', 'Jul', 'Aug', 'Sep')

In [ ]:
trajs_659['mes'] = trajs_659['data'].dt.strftime('%b')
trajs_659.head()

In [ ]:
periodo_umido = trajs_659.loc[trajs_659['mes'].isin(meses_chuvosos)]
periodo_umido.head()

In [ ]:
periodo_seco = trajs_659.loc[trajs_659['mes'].isin(meses_secos)]
periodo_seco.head()

### Velocidade

In [ ]:
gp_vel_umido = periodo_umido.groupby(['direcao_cat', 'velocidade_cat']).size().reset_index(name='frequencia')
gp_vel_umido.head()

In [ ]:
fig = vel_plot(gp_vel_umido, title='<b>Período úmido</b>')
fig.show()

In [ ]:
gp_vel_seco = periodo_seco.groupby(['direcao_cat', 'velocidade_cat']).size().reset_index(name='frequencia')
gp_vel_seco.head()

In [ ]:
fig = vel_plot(gp_vel_seco, title='<b>Período seco</b>')
fig.show()

### Chuva

In [ ]:
gp_chuva_umido = periodo_umido.groupby(['direcao_cat', 'evt_soma_chuva_cat']).size().reset_index(name='frequencia')
gp_chuva_umido.head()

In [ ]:
fig = chuva_plot(dataframe=gp_chuva_umido, title='<b>Período úmido</b>')
fig.show()

In [ ]:
gp_chuva_seco = periodo_seco.groupby(['direcao_cat', 'evt_soma_chuva_cat']).size().reset_index(name='frequencia')
gp_chuva_seco.head()

In [ ]:
fig = chuva_plot(dataframe=gp_chuva_seco, title='<b>Período seco</b>')
fig.show()

## Campanhas

In [ ]:
campanhas = trajs_659['campanha'].unique()

### Velocidade

In [ ]:
for campanha in campanhas:
    df = trajs_659.loc[trajs_659["campanha"] == campanha]
    df = df.groupby(['direcao_cat', 'velocidade_cat']).size().reset_index(name='frequencia')

    fig = vel_plot(df, title=f'<b>{campanha}</b>')
    #fig.show() 

### Chuva

In [ ]:
for campanha in campanhas:
    df = trajs_659.loc[trajs_659["campanha"] == campanha]
    df = df.groupby(['direcao_cat', 'evt_soma_chuva_cat']).size().reset_index(name='frequencia')
    
    fig = chuva_plot(df, title=f'<b>{campanha}</b>', template='plotly_dark', annotation_color='white')
    fig.show()